In [ ]:
import sys 
import torch
import cv2


sys.path.append("/home/hanglok/work/LightGlue")
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
from sam_utils import select_point,segment,mask_filter



def find_glue_points(image_array1,image_array2):
    '''
    args:
    image_array1: image array of the first image
    image_array2: image array of the second image

    returns:
    m_kpts0: matched keypoints of the first image
    m_kpts1: matched keypoints of the second image
    
    
    '''
    image_array1 = cv2.resize(image_array1, (320,240))
    image_array2 = cv2.resize(image_array2, (320,240))



    torch.set_grad_enabled(False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

    extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
    matcher = LightGlue(features="superpoint").eval().to(device)


    points = select_point(image_array1,input_is_image = True)
    masks = segment(image_array1, points,input_is_image = True)

    image0 = load_image(image_array1,input_is_image = True)
    image1 = load_image(image_array2,input_is_image = True)

    # print(image1.shape)

    feats0 = extractor.extract(image0.to(device))
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})



    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension

    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    # print("kpts0",kpts0,kpts0.shape)
    # print("kpts1",kpts1,kpts1.shape)
    # print("match",matches,matches.shape)

    matches = mask_filter(masks,matches,kpts0)
    # print(len(matches))

    # print("matches",matches)

    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]


    print("mkpts0",m_kpts0)
    print("mkpts1",m_kpts1)


    axes = viz2d.plot_images([image0, image1])
    viz2d.plot_matches(m_kpts0, m_kpts1, color="lime", lw=0.2)
    viz2d.add_text(0, f'Stop after {matches01["stop"]} layers', fs=20)


    return m_kpts0,m_kpts1


In [ ]:
image_path1 = "data/images-20241112-172123/camera1_rgb_0.jpg"
image_path2 = "data/images-20241112-172123/camera1_rgb_1.jpg"
image_array1 = cv2.imread(image_path1)
image_array2 = cv2.imread(image_path2)
find_glue_points(image_array1,image_array2)